In [ ]:
"""
# Resumo e conclusão

# prompt: me de o comando para carregar o drive

from google.colab import drive
drive.mount('/content/drive')

📓 1_Configuracao_Ambiente/
# Verificar o tipo de GPU disponível
!nvidia-smi
# Verificar a versão do Python e ambiente atual
import sys
print(f"Versão do Python: {sys.version}")
!pip --version
# Instalação de bibliotecas essenciais para processamento geoespacial
!pip install -U geopandas rasterio pyproj shapely fiona rtree pycrs mapclassify pyogrio
!pip install -U osmnx networkx folium keplergl
!pip install -U rasterstats xarray rioxarray netCDF4 
!pip install -U scipy scikit-learn statsmodels plotly matplotlib seaborn
!pip install -U tqdm jupyterlab notebook ipywidgets
!pip install -U pygeos contextily pysal momepy
!pip install -U psycopg2-binary sqlalchemy geoalchemy2
# Instalação das bibliotecas RAPIDS para processamento acelerado por GPU
# O L4 é compatível com RAPIDS, que acelera significativamente operações geoespaciais
!pip install -U cudf cuml cuspatial cupy
# Bibliotecas específicas para manipulação de dados tabulares e leitura de arquivos
!pip install -U pandas numpy openpyxl xlrd xlwt xlsxwriter
!pip install -U pyshp geojson topojson geobuf mapbox-vector-tile
!pip install -U geoviews holoviews datashader panel hvplot
!pip install -U dask distributed h5py hdf5 zarr
# Verificar importações essenciais e configurações básicas
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
import networkx as nx

print("Verificação de configuração básica concluída. Bibliotecas principais carregadas com sucesso.")
# Configurar ambiente para trabalhar com o L4 GPU
import os

# Definir variáveis de ambiente para otimização de bibliotecas geoespaciais
os.environ['USE_PYGEOS'] = '0'  # Usar GeoPandas com GEOS (mais estável)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Utilizar GPU 0

# Configurações para utilizar memória de GPU de forma eficiente
os.environ['RAPIDS_NO_INITIALIZE'] = '1'  # Inicialização manual do RAPIDS

print("Variáveis de ambiente configuradas para otimização com GPU L4")
# Configuração para visualização interativa
from IPython.display import display, HTML

# Aumentar limite de exibição para melhor visualização de dados geoespaciais
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# Configurar matplotlib para visualizações de alta qualidade
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100

print("Configurações de visualização otimizadas")
# Função utilitária para verificar consumo de memória
def check_memory_usage():
    """Exibe o uso atual de memória RAM e GPU."""
    print("Uso de memória RAM:")
    !free -h
    
    print("\nUso de memória GPU:")
    !nvidia-smi --query-gpu=memory.used,memory.total --format=csv
    
check_memory_usage()
# Testar leitura básica de dados geoespaciais
try:
    # Criar um GeoDataFrame simples para teste
    from shapely.geometry import Point
    
    # Criar alguns pontos de teste
    geometry = [Point(0, 0), Point(1, 1), Point(2, 2)]
    gdf = gpd.GeoDataFrame(geometry=geometry)
    
    # Plotar para verificar funcionamento
    gdf.plot()
    plt.title("Teste de Plotagem Geoespacial")
    plt.show()
    
    print("Teste de funcionalidade geoespacial concluído com sucesso!")
except Exception as e:
    print(f"Erro ao testar funcionalidade geoespacial: {e}")
# Resumo da configuração do ambiente
print("="*80)
print("CONFIGURAÇÃO DO AMBIENTE CONCLUÍDA COM SUCESSO")
print("="*80)
print("\nBibliotecas instaladas e configuradas:")
print("- Processamento geoespacial: GeoPandas, Rasterio, Shapely, Fiona")
print("- Análise de redes: NetworkX, OSMnx")
print("- Processamento de dados: Pandas, NumPy, SciPy, Scikit-learn")
print("- Visualização: Matplotlib, Folium, Plotly, Kepler.gl")
print("- Aceleração GPU: RAPIDS (cuDF, cuSpatial)")
print("\nPróximos passos:")
print("1. Execute 1.2_Carregamento_Datasets.ipynb para carregar os dados")
print("2. Execute 1.3_Configuracao_Sistema_Coordenadas.ipynb para configurar os sistemas de referência")
print("="*80)
1.2_Carregamento_Datasets.ipynb
# Carregamento de Datasets para Integração Geoespacial
# Este notebook carrega os datasets GPKG necessários para o projeto

import os
import glob
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import json
from shapely.geometry import box

# Montando o Google Drive para acessar os dados
from google.colab import drive
drive.mount('/content/drive')
# Definição do diretório onde estão os datasets
data_dir = '/content/drive/MyDrive/geoprocessamento_gnn/DATA'

# Verificando se o diretório existe
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"O diretório {data_dir} não foi encontrado. Verifique o caminho.")
    
print(f"Diretório de dados encontrado: {data_dir}")
# Listando todos os arquivos GPKG no diretório de dados
gpkg_files = glob.glob(os.path.join(data_dir, "*.gpkg"))
print(f"Encontrados {len(gpkg_files)} arquivos GPKG:")
for i, file in enumerate(gpkg_files):
    print(f"{i+1}. {os.path.basename(file)}")
    
if len(gpkg_files) == 0:
    print("Nenhum arquivo GPKG encontrado. Verifique o caminho ou extensão dos arquivos.")
# Função para explorar as camadas dentro de um arquivo GPKG
def explore_gpkg(gpkg_path):
    """
    Explora as camadas disponíveis em um arquivo GeoPackage.
    
    Args:
        gpkg_path: Caminho para o arquivo GPKG
    
    Returns:
        Um DataFrame com informações sobre as camadas
    """
    # Listar todas as camadas no arquivo
    layers = fiona.listlayers(gpkg_path)
    
    # Coletar informações sobre cada camada
    layer_info = []
    for layer in layers:
        try:
            # Abrir a camada para obter informações
            with fiona.open(gpkg_path, layer=layer) as src:
                # Obter a contagem de feições
                count = len(src)
                # Obter o tipo de geometria
                if count > 0:
                    geometry_type = src.schema['geometry']
                else:
                    geometry_type = "Desconhecido"
                # Obter o CRS
                crs = src.crs
                # Obter os campos de atributos
                fields = list(src.schema['properties'].keys())
                
                layer_info.append({
                    'Layer': layer,
                    'Feature Count': count,
                    'Geometry Type': geometry_type,
                    'CRS': crs,
                    'Fields': fields[:5] + ['...'] if len(fields) > 5 else fields
                })
        except Exception as e:
            print(f"Erro ao processar camada {layer}: {e}")
            layer_info.append({
                'Layer': layer,
                'Error': str(e)
            })
    
    return pd.DataFrame(layer_info)

# Importamos fiona aqui para listar as camadas
import fiona

# Explorando cada arquivo GPKG
for gpkg_file in gpkg_files:
    file_name = os.path.basename(gpkg_file)
    print(f"\n{'='*80}\nExplorando arquivo: {file_name}\n{'='*80}")
    
    layers_info = explore_gpkg(gpkg_file)
    display(layers_info)
# Função para carregar e armazenar todos os datasets em um dicionário
def load_all_datasets(gpkg_files):
    """
    Carrega todos os arquivos GPKG e suas camadas em um dicionário.
    
    Args:
        gpkg_files: Lista de caminhos para arquivos GPKG
    
    Returns:
        Um dicionário de GeoDataFrames organizados por nome de arquivo e camada
    """
    datasets = {}
    
    for gpkg_file in tqdm(gpkg_files, desc="Carregando arquivos GPKG"):
        file_name = os.path.basename(gpkg_file)
        datasets[file_name] = {}
        
        # Listar camadas
        layers = fiona.listlayers(gpkg_file)
        
        for layer in tqdm(layers, desc=f"Camadas em {file_name}", leave=False):
            try:
                # Carregar o GeoDataFrame
                gdf = gpd.read_file(gpkg_file, layer=layer)
                
                # Armazenar no dicionário
                datasets[file_name][layer] = gdf
                
                print(f"Carregado: {file_name} - {layer} - {len(gdf)} registros")
            except Exception as e:
                print(f"Erro ao carregar camada {layer} de {file_name}: {e}")
    
    return datasets

# Carregando todos os datasets
datasets = load_all_datasets(gpkg_files)

# Salvando em uma variável global para acesso posterior
import builtins
builtins.datasets = datasets

print("\nCarregamento completo. Todos os datasets estão armazenados na variável global 'datasets'.")
# Função para visualizar a extensão geográfica de todos os datasets em um mapa
def plot_dataset_extents(datasets):
    """
    Visualiza a extensão geográfica de todos os datasets carregados.
    
    Args:
        datasets: Dicionário de datasets carregados
    """
    # Criar um gráfico vazio
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))
    
    # Cores para diferentes datasets
    colors = plt.cm.tab20.colors
    color_idx = 0
    
    # Legenda
    legend_items = []
    
    # Iterar através dos datasets
    for file_name, layers in datasets.items():
        for layer_name, gdf in layers.items():
            # Verificar se o GeoDataFrame tem geometria
            if gdf.geometry.is_empty.all():
                print(f"Geometria vazia em {file_name} - {layer_name}")
                continue
            
            try:
                # Obter a extensão do dataset
                minx, miny, maxx, maxy = gdf.total_bounds
                extent_box = box(minx, miny, maxx, maxy)
                
                # Criar um GeoDataFrame com a extensão
                extent_gdf = gpd.GeoDataFrame(geometry=[extent_box], crs=gdf.crs)
                
                # Plotar no mapa
                color = colors[color_idx % len(colors)]
                extent_gdf.boundary.plot(ax=ax, color=color, linewidth=2, 
                                        label=f"{file_name} - {layer_name}")
                
                # Adicionar à legenda
                legend_items.append(f"{file_name} - {layer_name}")
                
                # Incrementar índice de cor
                color_idx += 1
            except Exception as e:
                print(f"Erro ao plotar extensão de {file_name} - {layer_name}: {e}")
    
    # Configurar o gráfico
    ax.set_title("Extensão geográfica dos datasets carregados", fontsize=16)
    ax.legend(legend_items, fontsize=10, loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()

# Visualizando a extensão dos datasets
plot_dataset_extents(datasets)
# Salvando o estado dos datasets para uso no próximo notebook
import pickle

# Criando pasta de estado se não existir
state_dir = os.path.join(data_dir, 'state')
os.makedirs(state_dir, exist_ok=True)

# Caminho para o arquivo de estado
state_file = os.path.join(state_dir, 'datasets_state.pkl')

# Salvando o dicionário de datasets
with open(state_file, 'wb') as f:
    pickle.dump(datasets, f)

print(f"Estado dos datasets salvo em: {state_file}")
print("Os datasets estão prontos para o próximo notebook de configuração do sistema de coordenadas.")
# Resumo do carregamento de dados
print("\n" + "="*80)
print("RESUMO DE CARREGAMENTO DE DADOS")
print("="*80)

# Contabilizar totais
total_layers = 0
total_features = 0

# Exibir estatísticas por arquivo
for file_name, layers in datasets.items():
    layer_count = len(layers)
    total_layers += layer_count
    
    feature_counts = [len(gdf) for gdf in layers.values()]
    total_file_features = sum(feature_counts)
    total_features += total_file_features
    
    print(f"\nArquivo: {file_name}")
    print(f"  Número de camadas: {layer_count}")
    print(f"  Total de feições: {total_file_features:,}")
    
    # Listar cada camada com contagem
    for layer_name, gdf in layers.items():
        print(f"    - {layer_name}: {len(gdf):,} feições")

print("\n" + "="*80)
print(f"TOTAL GERAL: {total_layers} camadas, {total_features:,} feições")
print("="*80)
1.3_Configuracao_Sistema_Coordenadas.ipynb
# Configuração do Sistema de Coordenadas para Integração Geoespacial
# Este notebook padroniza todos os datasets para o sistema SIRGAS 2000 23S (EPSG:31983)

import os
import pickle
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from shapely.geometry import box
import warnings

# Montando o Google Drive para acessar os dados
from google.colab import drive
drive.mount('/content/drive')
# Diretório de dados
data_dir = '/content/drive/MyDrive/geoprocessamento_gnn/DATA'
state_dir = os.path.join(data_dir, 'state')
state_file = os.path.join(state_dir, 'datasets_state.pkl')

# Verificando se o arquivo de estado existe
if not os.path.exists(state_file):
    raise FileNotFoundError(f"Arquivo de estado não encontrado: {state_file}")

# Carregando o estado dos datasets do notebook anterior
with open(state_file, 'rb') as f:
    datasets = pickle.load(f)

# Definindo globalmente para uso posterior
import builtins
builtins.datasets = datasets

print(f"Datasets carregados com sucesso do arquivo: {state_file}")
# Definição do sistema de coordenadas alvo: SIRGAS 2000 / UTM zone 23S (EPSG:31983)
TARGET_CRS = "EPSG:31983"

# Informações sobre o sistema de coordenadas alvo
sirgas_info = {
    'EPSG': 31983,
    'Nome': 'SIRGAS 2000 / UTM zone 23S',
    'Projeção': 'Transverse Mercator',
    'Datum': 'SIRGAS 2000',
    'Unidade': 'metros',
    'Zona UTM': '23S',
    'Meridiano Central': '-45',
    'Latitude de Origem': '0',
    'Falso Leste': '500000',
    'Falso Norte': '10000000',
    'Fator de Escala': '0.9996'
}

print("Sistema de coordenadas alvo:")
for key, value in sirgas_info.items():
    print(f"  {key}: {value}")

print("\nEste sistema de coordenadas é ideal para o nosso projeto porque:")
print("1. Mantém a precisão das medições em metros para a área de estudo")
print("2. É o sistema padrão para projetos geoespaciais no Brasil, especialmente na zona 23S")
print("3. Permite calcular áreas, distâncias e análises espaciais com precisão")
print("4. Minimiza distorções para a região de estudo")
# Função para verificar e tabular os sistemas de coordenadas atuais de todos os datasets
def check_crs(datasets):
    """
    Verifica os sistemas de coordenadas de todos os datasets.
    
    Args:
        datasets: Dicionário de datasets carregados
    
    Returns:
        DataFrame com informações sobre os sistemas de coordenadas
    """
    crs_info = []
    
    for file_name, layers in datasets.items():
        for layer_name, gdf in layers.items():
            try:
                # Obter informações do CRS
                if gdf.crs is None:
                    crs_code = None
                    crs_name = "Não definido"
                    is_projected = False
                    units = "Desconhecido"
                else:
                    crs_code = gdf.crs.to_epsg()
                    crs_name = gdf.crs.name if hasattr(gdf.crs, 'name') else str(gdf.crs)
                    is_projected = gdf.crs.is_projected
                    units = gdf.crs.axis_info[0].unit_name if hasattr(gdf.crs, 'axis_info') else "Desconhecido"
                
                needs_transformation = crs_code != 31983 and crs_code is not None
                
                crs_info.append({
                    'Arquivo': file_name,
                    'Camada': layer_name,
                    'EPSG': crs_code,
                    'CRS Nome': crs_name,
                    'Projetado': is_projected,
                    'Unidades': units,
                    'Requer Transformação': needs_transformation
                })
            except Exception as e:
                print(f"Erro ao verificar CRS de {file_name} - {layer_name}: {e}")
                crs_info.append({
                    'Arquivo': file_name,
                    'Camada': layer_name,
                    'EPSG': None,
                    'CRS Nome': f"ERRO: {str(e)}",
                    'Projetado': None,
                    'Unidades': None,
                    'Requer Transformação': None
                })
    
    return pd.DataFrame(crs_info)

# Analisando os sistemas de coordenadas atuais
crs_status = check_crs(datasets)
display(crs_status)

# Contabilizando quantos datasets precisam de transformação
requires_transformation = crs_status['Requer Transformação'].sum()
total_layers = len(crs_status)

print(f"\nStatus de transformação: {requires_transformation} de {total_layers} camadas precisam ser transformadas para SIRGAS 2000 / UTM zone 23S (EPSG:31983)")
# Função para transformar datasets para o CRS alvo
def transform_datasets(datasets, target_crs=TARGET_CRS):
    """
    Transforma todos os datasets para o sistema de coordenadas alvo.
    
    Args:
        datasets: Dicionário de datasets carregados
        target_crs: Sistema de coordenadas alvo
    
    Returns:
        Dicionário de datasets transformados
    """
    transformed_datasets = {}
    transformation_report = []
    
    # Para cada arquivo
    for file_name, layers in tqdm(datasets.items(), desc="Transformando arquivos"):
        transformed_datasets[file_name] = {}
        
        # Para cada camada
        for layer_name, gdf in tqdm(layers.items(), desc=f"Camadas em {file_name}", leave=False):
            try:
                # Verificar se o CRS atual é None
                if gdf.crs is None:
                    print(f"AVISO: {file_name} - {layer_name} não possui CRS definido. Atribuindo o CRS alvo sem transformação.")
                    gdf.crs = target_crs
                    transformed_gdf = gdf
                    status = "Atribuído CRS (sem transformação)"
                
                # Verificar se já está no CRS alvo
                elif gdf.crs.to_epsg() == 31983:
                    print(f"{file_name} - {layer_name} já está no CRS alvo. Nenhuma transformação necessária.")
                    transformed_gdf = gdf
                    status = "Já no CRS alvo"
                
                # Realizar a transformação
                else:
                    # Registrar informações antes da transformação
                    if 'geometry' in gdf.columns and len(gdf) > 0 and not gdf.geometry.is_empty.all():
                        pre_bounds = gdf.total_bounds
                    else:
                        pre_bounds = None
                    
                    # Executar a transformação
                    with warnings.catch_warnings():
                        warnings.filterwarnings("ignore", category=UserWarning)
                        transformed_gdf = gdf.to_crs(target_crs)
                    
                    # Registrar informações após a transformação
                    if pre_bounds is not None and len(transformed_gdf) > 0 and not transformed_gdf.geometry.is_empty.all():
                        post_bounds = transformed_gdf.total_bounds
                        status = "Transformado com sucesso"
                    else:
                        post_bounds = None
                        status = "Transformado (sem geometria para validar)"
                
                # Armazenar o GeoDataFrame transformado
                transformed_datasets[file_name][layer_name] = transformed_gdf
                
                # Registrar no relatório
                transformation_report.append({
                    'Arquivo': file_name,
                    'Camada': layer_name,
                    'CRS Original': str(gdf.crs),
                    'CRS Final': str(transformed_gdf.crs),
                    'Status': status,
                    'Registros': len(gdf)
                })
                
            except Exception as e:
                print(f"ERRO ao transformar {file_name} - {layer_name}: {e}")
                # Manter o dataset original em caso de erro
                transformed_datasets[file_name][layer_name] = gdf
                transformation_report.append({
                    'Arquivo': file_name,
                    'Camada': layer_name,
                    'CRS Original': str(gdf.crs) if hasattr(gdf, 'crs') else "Desconhecido",
                    'CRS Final': "ERRO",
                    'Status': f"Erro: {str(e)}",
                    'Registros': len(gdf) if hasattr(gdf, 'len') else "Desconhecido" 
                })
    
    return transformed_datasets, pd.DataFrame(transformation_report)

# Executando a transformação
transformed_datasets, transformation_report = transform_datasets(datasets)

# Atualizando a variável global com os datasets transformados
builtins.datasets = transformed_datasets

# Exibindo o relatório de transformação
display(transformation_report)
# Validação final: verificando se todos os datasets estão no sistema de coordenadas correto
def validate_transformations(datasets, target_crs=TARGET_CRS):
    """
    Verifica se todos os datasets estão no sistema de coordenadas alvo.
    
    Args:
        datasets: Dicionário de datasets transformados
        target_crs: Sistema de coordenadas alvo
    
    Returns:
        DataFrame com resultados da validação
    """
    validation_results = []
    target_epsg = 31983  # EPSG para SIRGAS 2000 / UTM zone 23S
    
    for file_name, layers in datasets.items():
        for layer_name, gdf in layers.items():
            try:
                current_epsg = gdf.crs.to_epsg()
                is_valid = current_epsg == target_epsg
                
                validation_results.append({
                    'Arquivo': file_name,
                    'Camada': layer_name,
                    'EPSG Atual': current_epsg,
                    'EPSG Alvo': target_epsg,
                    'Validação': "Sucesso" if is_valid else "Falha",
                    'CRS': str(gdf.crs)
                })
            except Exception as e:
                validation_results.append({
                    'Arquivo': file_name,
                    'Camada': layer_name,
                    'EPSG Atual': None,
                    'EPSG Alvo': target_epsg,
                    'Validação': f"Erro: {str(e)}",
                    'CRS': str(gdf.crs) if hasattr(gdf, 'crs') else "Desconhecido"
                })
    
    return pd.DataFrame(validation_results)

# Validando todas as transformações
validation_results = validate_transformations(transformed_datasets)
display(validation_results)

# Verificando sucesso geral
success_count = (validation_results['Validação'] == "Sucesso").sum()
total_count = len(validation_results)
success_rate = success_count / total_count * 100

print(f"\nValidação concluída: {success_count} de {total_count} camadas ({success_rate:.2f}%) estão no sistema de coordenadas SIRGAS 2000 / UTM zone 23S (EPSG:31983)")

if success_count < total_count:
    print("\nATENÇÃO: Algumas camadas não foram transformadas corretamente. Verifique o relatório acima.")
else:
    print("\nTodos os datasets foram transformados com sucesso para o sistema de coordenadas alvo!")
# Salvando os datasets transformados
import pickle

# Criando pasta de estado se não existir
state_dir = os.path.join(data_dir, 'state')
os.makedirs(state_dir, exist_ok=True)

# Caminho para o arquivo de estado
transformed_state_file = os.path.join(state_dir, 'datasets_transformed_state.pkl')

# Salvando o dicionário de datasets transformados
with open(transformed_state_file, 'wb') as f:
    pickle.dump(transformed_datasets, f)

print(f"Estado dos datasets transformados salvo em: {transformed_state_file}")
# Visualizando algumas feições para verificar visualmente a transformação
def plot_sample_geometries(datasets):
    """
    Plota amostras de geometrias para verificação visual.
    
    Args:
        datasets: Dicionário de datasets transformados
    """
    # Criar um layout de 2x2 para visualização
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    
    # Contador para controlar quantas camadas plotar
    plot_count = 0
    max_plots = 4
    
    # Cores para diferentes datasets
    colors = plt.cm.tab10.colors
    
    # Para cada arquivo e camada
    for file_name, layers in datasets.items():
        for layer_name, gdf in layers.items():
            if plot_count >= max_plots:
                break
                
            try:
                # Verificar se há geometrias para plotar
                if 'geometry' not in gdf.columns or len(gdf) == 0 or gdf.geometry.is_empty.all():
                    continue
                    
                # Plotar apenas uma amostra (máximo 100 feições)
                sample_size = min(100, len(gdf))
                ax = axes[plot_count]
                
                # Plotar a amostra
                gdf.sample(sample_size).plot(
                    ax=ax, 
                    color=colors[plot_count % len(colors)],
                    alpha=0.7,
                    edgecolor='black',
                    linewidth=0.5
                )
                
                # Configurar o título e rótulos
                ax.set_title(f"{file_name}\n{layer_name}", fontsize=12)
                ax.set_xlabel("Coordenada X (metros)")
                ax.set_ylabel("Coordenada Y (metros)")
                ax.grid(True)
                
                # Adicionar informações de CRS
                ax.text(0.5, -0.1, f"CRS: {gdf.crs.name if hasattr(gdf.crs, 'name') else str(gdf.crs)[:50]}...",
                       horizontalalignment='center', verticalalignment='center',
                       transform=ax.transAxes, fontsize=10)
                
                plot_count += 1
                
            except Exception as e:
                print(f"Erro ao plotar {file_name} - {layer_name}: {e}")
                continue
    
    if plot_count == 0:
        print("Nenhuma geometria disponível para visualização.")
        plt.close(fig)
        return
    
    # Ocultar eixos não utilizados
    for i in range(plot_count, max_plots):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

# Visualizando amostras das geometrias transformadas
plot_sample_geometries(transformed_datasets)
# Resumo e conclusão
print("="*80)
print("RESUMO DA CONFIGURAÇÃO DE SISTEMA DE COORDENADAS")
print("="*80)
print(f"\nSistema de coordenadas alvo: SIRGAS 2000 / UTM zone 23S (EPSG:31983)")
print(f"Total de arquivos processados: {len(transformed_datasets)}")
print(f"Total de camadas processadas: {total_count}")
print(f"Camadas transformadas com sucesso: {success_count} ({success_rate:.2f}%)")

print("\nBenefícios da padronização:")
print("1. Todas as análises espaciais agora usarão o mesmo sistema de referência")
print("2. Cálculos de área, distância e proximidade serão precisos e consistentes")
print("3. Visualizações de mapas estarão corretamente alinhadas")
print("4. Operações de sobreposição espacial funcionarão corretamente")

print("\nPróximos passos:")
print("1. Prosseguir para a integração de Edifícios e Uso do Solo")
print("2. Criar análises demográficas usando os datasets transformados")
print("3. Realizar as operações hidrográficas com os dados padronizados")

print("\nOs datasets transformados estão disponíveis na variável global 'datasets'")
print("="*80)
""" 